In [2]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [3]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

ModuleNotFoundError: No module named 'object_detection'

In [5]:
os.chdir("research")

In [13]:
!pip install tf_slim

In [6]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [7]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [8]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [9]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [10]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [11]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [12]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = image_path
  image_np=cv2.cvtColor(image_np,cv2.COLOR_BGR2RGB)
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=4)
  
  image_np=cv2.cvtColor(image_np,cv2.COLOR_BGR2RGB)
  bicyc_detect(image_np, output_dict['detection_classes'], output_dict['detection_scores'], output_dict['detection_boxes'])  
  return image_np

In [13]:
!pip install opencv-python

In [15]:
import cv2

img=cv2.imread(r'C:\Users\regil t\Downloads\Its_a_dead.jpg')
img=show_inference(detection_model,img)
#img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#display(Image.fromarray(img))
cv2.imshow('IMG',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [74]:
!pip install pafy

In [14]:
!pip install youtube-dl

In [13]:
import pafy
#url   = "https://www.youtube.com/watch?v=kz2j3gi8dUU"  mundrovideo
url = 'https://www.youtube.com/watch?v=L1rH1F7Dk_c'
vid = pafy.new(url)
best  = vid.getbest(preftype='any', ftypestrict=True)

In [14]:
print(best)

normal:mp4@640x360


In [15]:
from datetime import datetime

def bicyc_detect(image, classes, scores, boxes):
    for i in range(100):
        time= datetime.now()
        dt_stng = time.strftime("%d_%m_%Y_%H_%M_%S")
        
        if (classes[i]=="2" and score[i]>0.6):
            f_nm = os.path.join('C:/Users/regil t/Desktop/iNeurone/cv projects/tfod', dt_stng+'.jpg')
            cv2.imwrite(f_nm, image)

In [16]:
import cv2

In [17]:
#video=cv2.VideoCapture(r'C:\Users\regil t\Downloads\output_free.mp4')
video=cv2.VideoCapture(best.url)
while(True):
    ret,img=video.read()
    img=show_inference(detection_model,img)
    cv2.imshow('LIVE',img)
    k=cv2.waitKey(1)
    
    if(k==27):
        break
cv2.destroyAllWindows()